In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException 
import numpy as np

In [2]:
#Key 

#ulta_1 = moisturizes 
#ulta_2 = sunscreen 
#etc

In [3]:
### Get Chrome Driver Path 
path = r"C:\Users\avach\Downloads\clone\info2950\chromedriver.exe" 
driver = webdriver.Chrome(path) 

### Open Ulta Moisturizer Site
#driver.get('https://www.ulta.com/skin-care-moisturizers?N=2796Z1z13p3l')
#driver.get('https://www.ulta.com/skin-care-face-moisturizer?N=27h9') 
driver.get('https://www.ulta.com/skin-care-face-moisturizer?N=27h9&No=96&Nrpp=96')




In [4]:
all_prod = driver.find_element_by_xpath('//*[@id="product-category-cont"]')

html_list = all_prod.find_element_by_id("foo16")
all_li = html_list.find_elements_by_tag_name("li")

product_name = []
subpageurl = []


#append links for products to subpageurl list
for li in all_li: 
    first_atag = li.find_element_by_tag_name('a')
    product_links= first_atag.get_attribute('href')
    
#popup link, got rid of it 
    if product_links != 'http://www.ulta.com/ulta/creditcards/landingpage.jsp':
        subpageurl.append(product_links)
        
#get name of hyperlink, not actual link 

#https://sqa.stackexchange.com/questions/37024/how-do-i-get-the-text-of-a-hyperlink-not-the-actual-link-using-selenium-and-py (innerlink source)
        cut_text = first_atag.get_attribute('innerHTML')
        find_alt = cut_text.find('alt="')
        name = cut_text[find_alt+5:]
        find_border = name.find('" border=')
        name2 = name[:find_border]
        if '+' in name2: 
            plus = name2.find('+') 
            name2 = name2[:plus]
            
        product_name.append(name2)
        
        
    

In [5]:
brand_list = []
product_list = [] 
price_list = [] 
rating_list = []
ingredient_list = [] 


In [6]:
##loop to get info on first page 

for link in subpageurl:
    driver.get(link)  

    ## Get brand 
    try:
        brand_list.append(driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[1]').text)
    except: 
        brand_list.append('NA')
    
    ### Get Product Name 
    try:
        product_list.append(driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[2]').text)
    except: 
        product_list.append('NA')
     ### Get Price 
    try:
        price_list.append(driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/div[3]').text)
    except: 
        product_list.append('NA') 
        
    ### Get Rating Number 
    try: 
        rating_list.append(driver.find_element_by_xpath('//*[@id="pr-review-snapshot"]/header/section/div/div[1]/div/div[1]/div').text)
    except: 
        rating_list.append('NA')

    ### Once on product page, toggle ingredients and get text
    try:
        toggle = driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]')
        toggle.click() 

        ingredient_list.append(driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]/div[2]/div/div/div').text) 
    except:
        ingredient_list.append('NA')



KeyboardInterrupt: 

In [8]:
rating_list

['3.6',
 '4.7',
 '4.5',
 '4.5',
 '4.7',
 '4.3',
 '4.7',
 '4.5',
 '4.3',
 '4',
 'NA',
 '4.5',
 '4.8',
 '4.2']

In [11]:
ulta_moisturizer_df = pd.DataFrame(
    {'brand': brand_list[:96],
     'name': product_list[:96],
     'price': price_list[:96],
     'rating': rating_list[:96],
     'ingredients': ingredient_list[:96]
    }) 

pd.set_option('display.max_rows',None)
ulta_moisturizer_df
#ulta_moisturizer_df_keep=ulta_moisturizer_df.copy()

,brand,name,price,rating,ingredients
0,Clinique,Clinique iD Dramatically Different For Lines &...,Price\n$39.50,3.6,Clinique iD Active Cartridge Concentrate for I...
1,Lancôme,Bienfait Multi-Vital SPF 30 Day Cream Moisturizer,Price\n$56.00,4.7,"Water, Dimethicone, Glycerin, Alcohol Denat., ..."
2,Neutrogena,Hydro Boost City Shield Water Gel Sunscreen SP...,Sale Price\n$23.19\nOriginal Price\n$28.99,4.5,"Active: Avobenzone 2%, Homosalate 4%, Octisala..."
3,Murad,Hydro-Dynamic Ultimate Moisture,Price\n$75.00,4.5,"Water (Aqua), C12-15 Alkyl Benzoate, Caprylic/..."
4,Murad,Intense Recovery Cream,Only at Ulta Beauty,4.7,"Water/Aqua/Eau, Ethylhexyl Palmitate, Betaine,..."
5,SUNDAY RILEY,Ice Ceramide Moisturizing Cream,Price\n$65.00,4.3,"Extract Blend [Water/Eau/Aqua, Chondrus Crispu..."
6,Curél,Intensive Moisture Facial Cream,Sale Price\n$21.00\nOriginal Price\n$30.00,4.7,"Water/Aqua, Glycerin, Cetyl-PG Hydroxyethyl Pa..."
7,Philosophy,Purity Made Simple Moisturizer,Price\n$28.00,4.5,"Aqua/Water/Eau, Cyclopentasiloxane, Bis-PEG-18..."
8,La Roche-Posay,Anthelios 50 Daily Face Primer with Sunscreen ...,Sale Price\n$31.99\nOriginal Price\n$39.99,4.3,"Active: Avobenzone 3% (Sunscreen), Homosalate ..."
9,GLAMGLOW,GLOWSTARTER Mega Illuminating Hyaluronic Acid ...,Price\n$50.00,4,"Water\Aqua\Eau, Dimethicone, Butylene Glycol, ..."


In [9]:
print(len(brand_list))
print(len(product_list))
print(len(price_list))
print(len(rating_list))
print(len(ingredient_list))

97
98
96
97
97


In [29]:
ulta_moisturizer_df_keep

,brand,name,price,rating,ingredients
0,It Cosmetics,Confidence In A Cream Anti-Aging Moisturizer,Price\n$49.00,4.6,"Aqua (Water, Eau), Butylene Glycol, Cyclopenta..."
1,Clinique,Dramatically Different Moisturizing Lotion+,Price\n$29.50,4.3,"Water / Aqua / Eau, Mineral Oil / Paraffinum L..."
2,Peter Thomas Roth,Water Drench Hyaluronic Cloud Cream Hydrating ...,Price\n$52.00,4.4,"Water/Aqua/Eau, Glycerin, Dipropylene Glycol, ..."
3,ELEMIS,Pro-Collagen Marine Cream,Price\n$128.00,4.5,"Aqua/Water/Eau, Glycerin, Caprylic/Capric Trig..."
4,StriVectin,SD Advanced Intensive Concentrate for Wrinkles...,Price\n$139.00,NA,"Aqua (Water, Eau), Glycerin, Pentaerythrityl T..."
5,Clinique,Dramatically Different Moisturizing Gel,Price\n$29.50,4.6,"Water / Aqua / Eau, Dimethicone, Isododecane, ..."
6,Shiseido,Essential Energy Moisturizing Cream,Price\n$48.00,NA,"Water(Aqua/Eau), Dimethicone, Dipropylene Glyc..."
7,Lancôme,Hydra Zen Anti-Stress Gel Cream,Price\n$48.00,NA,This item is not eligible for coupon discounts...
8,Murad,Essential-C Day Moisture Broad Spectrum SPF 30...,Price\n$65.00,4.5,"Active Ingredients: Octinoxate 7.5%, Homosalat..."
9,Murad,Nutrient-Charged Water Gel,Price\n$62.00,NA,"Water/Aqua/Eau, Dimethicone, Glycerin, Propane..."


In [27]:
brand_list

['It Cosmetics',
 'Clinique',
 'Peter Thomas Roth',
 'ELEMIS',
 'StriVectin',
 'Clinique',
 'Shiseido',
 'Lancôme',
 'Murad',
 'Murad',
 'PEACH & LILY',
 'Juice Beauty',
 'Philosophy',
 'Tula',
 'The Ordinary',
 'Origins',
 'Clinique',
 'First Aid Beauty',
 'CeraVe',
 'Perricone MD',
 'CeraVe',
 'CeraVe',
 "Kiehl's Since 1851",
 'Clinique',
 'It Cosmetics',
 'La Roche-Posay',
 'Dermalogica',
 'Philosophy',
 'Dermalogica',
 'CeraVe',
 'Clinique',
 'COSRX',
 'Neutrogena',
 'Dermalogica',
 'Clinique',
 'belif',
 'belif',
 'Philosophy',
 'Dermalogica',
 'Dermalogica',
 'La Roche-Posay',
 'No7',
 'First Aid Beauty',
 'La Roche-Posay',
 'First Aid Beauty',
 'Neutrogena',
 'First Aid Beauty',
 'Estée Lauder',
 'First Aid Beauty',
 'Lancôme',
 'Olay',
 'Pacifica',
 'e.l.f. Cosmetics',
 'No7',
 'Olay',
 'La Roche-Posay',
 'Tula',
 'CANNUKA',
 'Mario Badescu',
 'Dermalogica',
 'Beekman 1802',
 'Neutrogena',
 'Estée Lauder',
 'First Aid Beauty',
 'Mario Badescu',
 'Philosophy',
 'La Roche-Posay',

In [15]:
ulta_moisturizer_df.to_csv('ulta_m1.csv', index = False)

In [ ]:
### Find first product and click to open 
#prod = driver.find_element_by_xpath('//*[@id="xlsImpprod13641053"]/div[2]') 
#prod.click()

In [ ]:
### Get brand
# brand = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[1]').text
# print(brand)

# ### Get Product Name 
# name = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[2]').text
# print(name) 
 

# ### Get Rating Number
# star = driver.find_element_by_xpath('//*[@id="pr-review-snapshot"]/header/section/div/div[1]/div/div[1]/div').text
# print(star) 

# ### Once on product page, toggle ingredients and get text
# toggle = driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]')
# toggle.click() 

# ingredient = driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]/div[2]/div/div/div').text
# print(ingredient) 

# ### Get Price 
# price = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/div[3]').text
# print(price)

In [ ]:
# def scrollDown(driver, n_scroll):

#     body = driver.find_element_by_tag_name("body")
#     while n_scroll >= 0:
#         body.send_keys(Keys.PAGE_DOWN)
#         n_scroll -= 1
#     return driver

In [ ]:
##next steps 

#move into data frame 
#scrol through each of the 14 pages 


In [16]:
ulta_m1 = pd.read_csv('ulta_m1.csv')

In [17]:
ulta_m1.head()

,brand,name,price,rating,ingredients
0,Clinique,Clinique iD Dramatically Different For Lines &...,Price\n$39.50,3.6,Clinique iD Active Cartridge Concentrate for I...
1,Lancôme,Bienfait Multi-Vital SPF 30 Day Cream Moisturizer,Price\n$56.00,4.7,"Water, Dimethicone, Glycerin, Alcohol Denat., ..."
2,Neutrogena,Hydro Boost City Shield Water Gel Sunscreen SP...,Sale Price\n$23.19\nOriginal Price\n$28.99,4.5,"Active: Avobenzone 2%, Homosalate 4%, Octisala..."
3,Murad,Hydro-Dynamic Ultimate Moisture,Price\n$75.00,4.5,"Water (Aqua), C12-15 Alkyl Benzoate, Caprylic/..."
4,Murad,Intense Recovery Cream,Only at Ulta Beauty,4.7,"Water/Aqua/Eau, Ethylhexyl Palmitate, Betaine,..."
